---
toc: true
comments: false
layout: post
title: Passion Project Log Work
description: Log of work done and issues faced over the course of the project
type: tangibles
courses: { csp: {week: 10} }
---

Since brainstorming has already been outlined in the passion project plan, this notebook will not detail that. I've done both frontend and backend so this will be divided into two sections.

## Backend
The pillow library contains more than a few image manipulation functions. Entering this, I immediately thought that the most difficult part in the beginning would be to connect the frontend and backend. Despite that, I decided to create a testing function first for a pixelate function. I expected PIL to already have a function to do this, and hence pored over way too much of PIL documentation before realizing there wasn't one. However, all of this reading allowed me to better create a function to do so.

PIL Documentation: https://pillow.readthedocs.io/en/stable/handbook/index.html

In [ ]:
def pixelate(image, size):
    org_size = image.size
    pixelate_lvl = size

    image = image.resize(
        size=(org_size[0] // pixelate_lvl, org_size[1] // pixelate_lvl),
        resample=0)
    image = image.resize(org_size, resample=0)

    return image

def imageToBase64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue())
    img_str = img_str.decode('utf-8')
    return img_str

def base64toImage(base64string):
    img_str = base64.b64decode(base64string)
    image = Image.open(BytesIO(img_str))
    return image

The above pixelate function scales the image down first, by a factor of the selected pixelation size. PIL's resize function automatically averages out the pixels, so this was perfect for the job. I ultimately decided to keep the size of the image the same, although in the future a function could be implemented to do this as well.

Then, I created two functions for converting to and from base64 to a PIL image. These functions were not difficult to create and mostly involved research and the base64 library. However, I quickly ran into errors and with the help of more research, I learned the BytesIO library was necessary to use to convert.

After testing this function, I placed it inside an API:

In [ ]:
    class _Pixelate(Resource):
        def post(self):
            data = request.get_json()  # Get JSON data from the request
            pixelated_image = pixelate(base64toImage(data['base64image']), int(data['pixelate_level']))
            response = jsonify({"base64image": imageToBase64(pixelated_image)})
            createImage(data['image_name'], 'pixelate', imageToBase64(pixelated_image)) # adds to database
            return response

## Postman issues
I deployed a test function and the GET request worked fine, but when I ran to test this pixelate function, things quickly went awry. I realized that I should've incorporated the convert functions into my pixelate function, but I managed to catch some small errors. In the future and when I implemented the combine function later, I definitely would've done this. This was also when I learned what a PUT request was and why I shouldn't use it since I wouldn't want a whole image encoded in the URL. 

## Flask deployment
While trying to deploy to flask I ran into many issues. I had to re-read the instructions multiple times and learned that I wasn't testing properly with docker desktop. While testing with docker desktop, I kept running into a library not found error with PIL and after research I learned that I needed to put all my libraries into requirements.txt in order for them to be installed. I also learned that debugging errors without docker and running main.py instead is easier as that way errors are actually shown. Through this whole process though, I learned where to find logs and errors, and had a better understanding of this after.

## Database
I also worked on the combine function, but that was mostly similar to pixelate except it involved more debugging. The database is currently in progress.

## Frontend 